# **MENGGUNAKAN SVM**

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from skimage import exposure
from sklearn.model_selection import GridSearchCV


local = 'cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

'wget' is not recognized as an internal or external command,
operable program or batch file.
c:\Users\asus\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [15]:
# Load and preprocess the data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

for img_path in os.listdir(train_cats_dir):
    image_paths.append(os.path.join(train_cats_dir, img_path))
    labels.append(0)  # 0 represents cats

for img_path in os.listdir(train_dogs_dir):
    image_paths.append(os.path.join(train_dogs_dir, img_path))
    labels.append(1)  # 1 represents dogs

# Feature extraction (HOG)
def extract_hog_features(image_path):
    image = cv2.imread(image_path)
    # Resize the image to a consistent size
    image = cv2.resize(image, (150, 150))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, hog_image = hog(gray_image, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True)
    return features

# Extract HOG features for all images and convert to NumPy array
X = [extract_hog_features(img_path) for img_path in image_paths]
X = np.array(X)  # Convert the list to a NumPy array
y = labels

# Split data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid to search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 2.0, 10, 100, 1000],
    'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'degree' : [0, 1, 2, 3, 4, 5, 6]
}

# Create an SVM classifier
svm_classifier = svm.SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm_classifier, param_grid, cv=4)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Make predictions using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Precision: {precision * 100:.2f}%")
print(f"Test Recall: {recall * 100:.2f}%")
print(f"Test F1: {recall * 100:.2f}%")

KeyboardInterrupt: 

In [ ]:
# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Load and preprocess the validation data
validation_image_paths = []  # List of validation image file paths
validation_labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

for img_path in os.listdir(validation_cats_dir):
    validation_image_paths.append(os.path.join(validation_cats_dir, img_path))
    validation_labels.append(0)  # 0 represents cats

for img_path in os.listdir(validation_dogs_dir):
    validation_image_paths.append(os.path.join(validation_dogs_dir, img_path))
    validation_labels.append(1)  # 1 represents dogs

# Extract HOG features for all validation images and convert to NumPy array
X_validation = [extract_hog_features(img_path) for img_path in validation_image_paths]
X_validation = np.array(X_validation)  # Convert the list to a NumPy array
y_validation = validation_labels

# Use your trained SVM model to make predictions on the validation data
y_validation_pred = best_model.predict(X_validation)

# Calculate metrics for the validation data
accuracy_validation = accuracy_score(y_validation, y_validation_pred)
precision_validation = precision_score(y_validation, y_validation_pred)
recall_validation = recall_score(y_validation, y_validation_pred)
f1_validation = f1_score(y_validation, y_validation_pred)

print(f"Validation Accuracy: {accuracy_validation * 100:.2f}%")
print(f"Validation Precision: {precision_validation * 100:.2f}%")
print(f"Validation Recall: {recall_validation * 100:.2f}%")
print(f"Validation F1: {recall_validation * 100:.2f}%")

Validation Accuracy: 73.00%
Validation Precision: 73.47%
Validation Recall: 72.00%
Validation F1: 72.00%
